In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt



In [ ]:

def get_dft(img , shift=True):
    img_zeropad = np.zeros((img.shape[0]*2 , img.shape[1]*2))
    img_zeropad[:img.shape[0] , :img.shape[1]] = img
#     print(img_zeropad.shape)
    img_fourier = np.fft.fft2(img_zeropad)
    if shift:
        img_fourier_shift = np.fft.fftshift(img_fourier)
    else:
        return img_fourier
    return img_fourier_shift

def apply_filter_in_frequency_domain(inp_img , filtr):
    filtr2 = np.zeros(inp_img.shape)
    filtr2[:filtr.shape[0] , :filtr.shape[1]] = filtr # zero padding
#     print(filtr2)
    filter_in_fourier = np.fft.fft2(filtr2) # fft
    filter_in_fourier_shift = np.fft.fftshift(filter_in_fourier) # shift
    
    out_img_fourier = filter_in_fourier_shift * inp_img
    
    out_img_fourier_shift = np.fft.fftshift(out_img_fourier) #shift
    out_img = np.fft.ifft2(out_img_fourier_shift) # inverse fourier transfrom
    return out_img.real


In [ ]:
im1 = cv2.cvtColor(cv2.imread('../Images/4/Barbara.bmp'), cv2.COLOR_BGR2GRAY)
im1_fourier_shift = get_dft(im1)

In [ ]:
def apply_filter_and_save(img , filtr , name):
    filter_result = apply_filter_in_frequency_domain(img , filtr)
    filter_result_cut = filter_result[:im1.shape[0] ,:im1.shape[1] ]
    filter_result_cut *= (255/filter_result_cut.max())
    plt.imshow(filter_result_cut , cmap="gray")
#     print(filter_result_cut)
    cv2.imwrite(f'./fft_results/{name}.jpg' ,filter_result_cut )

# 4.1.1

In [ ]:
filtera1 = np.array([[1 , 2, 1]])
apply_filter_and_save(im1_fourier_shift , filtera1 , "filtera1")

filtera2 = np.array([[1] , [2], [1]])
apply_filter_and_save(im1_fourier_shift , filtera2 , "filtera2")

filterb = np.array([[-1 , -1 , -1] , [-1 , 8 , -1] , [-1 , -1 , -1]])
apply_filter_and_save(im1_fourier_shift , filterb , "filterb")


filterc = np.array([[0 , -1 , 0] , [-1 , 5 , -1] , [0 , -1 , 0]])

apply_filter_and_save(im1_fourier_shift , filterc , "filterc")

# 4.1.2

In [ ]:
lena_img = cv2.cvtColor(cv2.imread('../Images/4/Lena.bmp'), cv2.COLOR_BGR2GRAY)
f16_img = cv2.cvtColor(cv2.imread('../Images/4/F16.bmp'), cv2.COLOR_BGR2GRAY)
baboon_img = cv2.cvtColor(cv2.imread('../Images/4/Baboon.bmp'), cv2.COLOR_BGR2GRAY)

In [ ]:
images = []
images.append({"file":lena_img , "name":"lena"})
images.append({"file":f16_img , "name":"f16"})
images.append({"file":baboon_img , "name":"baboon"})

for img in images:
    img_dft_magnitude_noshift = np.abs(get_dft(img["file"] , False))
    cv2.imwrite(f'./fft_results/{img["name"]}_nolog_noshift.jpg' ,img_dft_magnitude_noshift)
    img_log_noshift = (np.log(img_dft_magnitude_noshift)*10).astype(np.uint8)
    print(img_log_noshift)
    cv2.imwrite(f'./fft_results/{img["name"]}_log_noshift.jpg' ,img_log_noshift)
    
    img_dft_magnitude = np.abs(get_dft(img["file"] , True))
    cv2.imwrite(f'./fft_results/{img["name"]}_nolog.jpg' ,img_dft_magnitude)
    img_log = (np.log(img_dft_magnitude)*10).astype(np.uint8)
    cv2.imwrite(f'./fft_results/{img["name"]}_log.jpg' ,img_log)
#     plt.imshow(baboon_dft_magnitude , cmap="gray")

# 4.2.2

In [ ]:
lena_dft = get_dft(lena_img , False)
T = 1/4
N = 1024
np.seterr(divide = 'ignore') 
filtr_type = "a"

if filtr_type == "a":
    lena_dft[int(T*N):int((1-T)*N) ,int(T*N):int((1-T)*N)] = 0 # a
else:
    lena_dft[0:int(T*N) , 0:int(T*N)] = 0 # b-i
    lena_dft[0:int(T*N) , int((1-T)*N):int(N-1)] = 0 # b-ii
    lena_dft[int((1-T)*N) :int(N-1) , 0:int(T*N)] = 0 # b-iii
    lena_dft[int((1-T)*N):N-1 , int((1-T)*N):N-1] = 0 # b-iv

spatial_lena = np.fft.ifft2(lena_dft)

spatial_lena = spatial_lena[:lena_img.shape[0] ,:lena_img.shape[1] ]

spatial_lena = np.abs(spatial_lena)


lena_log_noshift = (np.log(np.abs(lena_dft))*10).astype(np.uint8)
plt.imshow(lena_log_noshift , cmap="gray")
# plt.imshow(spatial_lena , cmap="gray")
cv2.imwrite(f'./fft_results/lenaT{T}-{filtr_type}.jpg' ,spatial_lena)
cv2.imwrite(f'./fft_results/lena_magn_T{T}-{filtr_type}.jpg' ,lena_log_noshift)